In this tutorial we are going to see how to use the Flywheel-sdk in python to interact with our projects and launch tasks. The Flywheel SDK is a toolbox that provides programmatic access to the Flywheel API endpoints.

This tutorial is based on the original Flywheel documentation:

https://docs.flywheel.io/hc/en-us

https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/index.html

First thing we need to do is to install the flywheel-sdk package using pip. Pip is a command to install packages and libraries from the Python Package Index (PyPI), a repository of software for Python (similar to CRAN and bioconductor repositories in R). To install pip, please refer to https://pip.pypa.io/en/stable/installing/

In [2]:
!pip install -U flywheel-sdk

Then, in order to communicate with our flywheel server, we need a client and a way to tell the server that it's us interacting with it. This is done through an API key, which can be found accessing your profile information in your flywheel account in https://bridge-center.flywheel.io/

![](https://drive.google.com/uc?export=view&id=11WcX19Y6ZvUJ4QbQZVtfIhjGeu_fnkeo)


In [3]:
# Import the flywheel-sdk package (Now, python runs "all" of the code in the package)
import flywheel

# Define your (mine in this case) key, that will allow us to communicate with the client
my_api_key =  'bridge-center.flywheel.io:GgTV6C9gYlMP0RhkmX'

# Define an instance of the client, that we saved it in a variable that we named "fw"
fw = flywheel.Client(my_api_key)

type(fw)

flywheel.client.Client

Now that we have defined our client, we can start communicating with our Flywheel server, to access the data, modify, run analyses, etc.

# BASIC STUFF

At this point, it's important to know that the data on the Flywheel server are organized as containers and these are ordered hierarchically. The order is as follows:

    1-Group

    2-Project

    3-Subject

    4-Session

    5-Acquisition

![](https://drive.google.com/uc?export=view&id=1AwFaT9xwcNLE3Qpl_ap1U-TjVxVG89sD)

In these five containers, the parent/child order is rigidly enforced. For example, a “Group” container cannot directly have an acquisition container attached as a child. Instead, it must have a “Project” child, which must have a “Subject” child, which must have a “Session” child, which then can have an “Acquisition” child. 

Each of these containers can have files, metadata information and analyses. **Analysis objects are another kind of containers**, as they've also got files and information.

**With the exception of Groups, all containers and objects within Flywheel have unique alphanumeric IDs that can be used to retrieve them so that we can interact with them (e.g. downloading data, changing meta fields information, etc).**

### Groups

First thing that we can do is to see all the groups that our client belongs to or has access to. This can be done by using the method *groups()* of our client.

In [ ]:
fw.groups

In [4]:
print(f'My client has access to {len(fw.groups())} groups')

My client has access to 4 groups


In [5]:
# This is one returned group object, which is similar to a dictionary (~ a structure array in Matlab)
fw.groups()[0]

{'created': datetime.datetime(2019, 6, 18, 15, 49, 17, 733000, tzinfo=tzlocal()),
 'editions': {'lab': False},
 'id': 'coax',
 'label': 'Verstynen-Lab',
 'modified': datetime.datetime(2019, 9, 24, 19, 13, 26, 188000, tzinfo=tzlocal()),
 'permissions': [{'access': 'admin', 'id': 'timothyv@andrew.cmu.edu'},
                 {'access': 'rw', 'id': 'kbond@andrew.cmu.edu'},
                 {'access': 'admin', 'id': 'jrasero.daparte@gmail.com'},
                 {'access': 'rw', 'id': 'alexisp@andrew.cmu.edu'},
                 {'access': 'ro', 'id': 'canakgun@flywheel.io'},
                 {'access': 'admin', 'id': 'timothyv@cmu.edu'},
                 {'access': 'rw', 'id': 'jraserod@andrew.cmu.edu'},
                 {'access': 'ro', 'id': 'yuanw3@andrew.cmu.edu'},
                 {'access': 'ro', 'id': 'rmadan@andrew.cmu.edu'},
                 {'access': 'rw', 'id': 'AIS36@pitt.edu'}],
 'permissions_template': [{'id': 'timothyv@andrew.cmu.edu',
                           'role_ids': 

In [6]:
# In this way, we can list and print all the groups that this client has access to
for group in fw.groups():
    print(f'Group Name/Label: {group.label}; Group ID: {group.id} ')

Group Name/Label: Verstynen-Lab; Group ID: coax 
Group Name/Label: Erickson-Lab; Group ID: kericksonlab 
Group Name/Label: Tarr-Lab; Group ID: mtarrlab 
Group Name/Label: Gianaros-Lab; Group ID: pgianaroslab 


As we'll be seeing below many times, once known the ID of the objects, we can retrieve them passing this ID to their method *.get()*. 

For example, let's say we want to operate on data belonging to Pete's lab, we can define an object of this group as follows

In [7]:
pete_lab = fw.get("pgianaroslab")
pete_lab

{'created': datetime.datetime(2019, 6, 18, 15, 37, 26, 958000, tzinfo=tzlocal()),
 'editions': {'lab': False},
 'id': 'pgianaroslab',
 'label': 'Gianaros-Lab',
 'modified': datetime.datetime(2020, 10, 7, 17, 58, 22, 521000, tzinfo=tzlocal()),
 'permissions': [{'access': 'admin', 'id': 'timothyv@andrew.cmu.edu'},
                 {'access': 'admin', 'id': 'rcesaro@andrew.cmu.edu'},
                 {'access': 'rw', 'id': 'tekraynak@gmail.com'},
                 {'access': 'rw', 'id': 'jaserod@andrew.cmu.edu'},
                 {'access': 'rw', 'id': 'jraserod@andrew.cmu.edu'},
                 {'access': 'ro', 'id': 'canakgun@flywheel.io'},
                 {'access': 'admin', 'id': 'timothyv@cmu.edu'},
                 {'access': 'admin', 'id': 'RLC66@PITT.EDU'},
                 {'access': 'admin', 'id': 'rlc66@pitt.edu'},
                 {'access': 'admin', 'id': 'RLC66@pitt.edu'},
                 {'access': 'rw', 'id': 'AIS36@pitt.edu'},
                 {'access': 'ro', 'id': 'BK

### Projects

Each group owns a certain research project (e.g. Pete's lab->NOAH). 

Once we have a Group object, we can list the projects that we have access to from that group using its method *.projects()*. For example, using the Pete's lab object defined previously:

In [8]:
print(f"I have access to {len(pete_lab.projects())} projects from  Pete's lab")

I have access to 2 projects from  Pete's lab


Using a nested loop, we can easily list all the projects from the Groups we have access to:

In [9]:
for group in fw.groups():
    for project in group.projects():
        print(f'Group Name/Label: {group.label}; Project Label: {project.label}; Project ID: {project.id}')

Group Name/Label: Verstynen-Lab; Project Label: LOKI1; Project ID: 5d1a15e20a578100367d3179
Group Name/Label: Erickson-Lab; Project Label: IGNITE; Project ID: 5d4aca01e3250a0037b76693
Group Name/Label: Erickson-Lab; Project Label: REACT; Project ID: 5d25ffbd19a9be0031eb0963
Group Name/Label: Erickson-Lab; Project Label: Unsorted; Project ID: 5d43190c52f58b003ac66645
Group Name/Label: Erickson-Lab; Project Label: IGNITE_phantoms; Project ID: 5d4d979ee3250a0033b767d4
Group Name/Label: Erickson-Lab; Project Label: eBACH; Project ID: 5d10ff90f7ee630031b225a7
Group Name/Label: Erickson-Lab; Project Label: AIM-IGNITE; Project ID: 5d4ac9e3e3250a0032b76610
Group Name/Label: Erickson-Lab; Project Label: eBACH_TRASH; Project ID: 5e457ce233070c059dd87941
Group Name/Label: Tarr-Lab; Project Label: LOKICAT; Project ID: 5d3f17c352f58b003ac65f57
Group Name/Label: Gianaros-Lab; Project Label: NOAH_TRASH; Project ID: 5e457cbb33070c059ad87937
Group Name/Label: Gianaros-Lab; Project Label: NOAH; Project 

At this point, we can load a project object and work with it using its ID through the function *get()* or *get_project()* of the client. As mentioned above, all containers and objects but Groups within Flywheel have unique IDs.

In [10]:
# for example, if we wanted to load a NOAH project object
noah_project = fw.get_project("5d10ff86f7ee630033b2261e")

# We could do the same using the function get() instead
#noah_project = fw.get("5d10ff86f7ee630033b2261e")

If we used this object, we'd be operating at the project level. Below this, we have our subjects.

### Subjects

In [11]:
print(f'NOAH project has {len(noah_project.subjects())} subjects')

NOAH project has 273 subjects


In [12]:
noah_project.subjects()[0]

{'age': None,
 'analyses': [],
 'code': 'N6019',
 'cohort': None,
 'created': datetime.datetime(2019, 7, 12, 19, 37, 41, 197000, tzinfo=tzlocal()),
 'ethnicity': None,
 'files': [],
 'firstname': None,
 'id': '5d28e18519a9be0033eb07aa',
 'info': {},
 'info_exists': False,
 'label': 'N6019',
 'lastname': None,
 'master_code': None,
 'mlset': None,
 'modified': datetime.datetime(2020, 10, 8, 19, 6, 21, 525000, tzinfo=tzlocal()),
 'notes': [],
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'pgianaroslab',
             'project': '5d10ff86f7ee630033b2261e',
             'session': None,
             'subject': None},
 'permissions': [],
 'project': '5d10ff86f7ee630033b2261e',
 'race': None,
 'revision': 3,
 'sex': None,
 'species': None,
 'strain': None,
 'tags': [],
 'type': None}

In [13]:
for subj in noah_project.subjects()[:10]:
    print(f'Subject label: {subj.label}, ID: {subj.id}')

Subject label: N6019, ID: 5d28e18519a9be0033eb07aa
Subject label: N6115, ID: 5d2f8a9d7288c50038d143ba
Subject label: N6180, ID: 5d66c1f5e3250a002eb76b7c
Subject label: N6176, ID: 5da89d7f119463001c08f8a3
Subject label: N6199, ID: 5db3069f119463002508fb74
Subject label: N6075, ID: 5db710d5119463002008f9f4
Subject label: P9001, ID: 5dcad87b119463002d093a8e
Subject label: N6301, ID: 5df94bbf26ab57002947e0d7
Subject label: N6321, ID: 5e1678d5bc1d310026c97d84
Subject label: N6104, ID: 5d309ebd7288c50038d1446e


### Sessions

In [14]:
subject_N6019 = fw.get("5d28e18519a9be0033eb07aa")

Now, we can go another level down in the subject object by looking into their sessions using the *sessions()* method.

In [15]:
print(f"this subject has {len(subject_N6019.sessions())} sessions")

this subject has 1 sessions


A same subject can have more than one session. That's what we have in eBACH, for example:

In [16]:
subject_E1021 = fw.get("5d24eb2319a9be0033eb0739")
print(f"Subject {subject_E1021.label} has {len(subject_E1021.sessions())} sessions")

Subject E1021 has 3 sessions


We can always access all sessions within a projects very easily using the same method. This can be useful if you want to launch some pipeline to all sessions at once.

In [17]:
print(f'NOAH project has {len(noah_project.sessions())} sessions')

NOAH project has 275 sessions


In [18]:
# Let's list the first 10 sessions
for ses in noah_project.sessions()[:10]:
    print("Subject label:", ses.subject.label, "Session label:", ses.label, " ID: ", ses.id)

Subject label: N6062 Session label: ses-01  ID:  5d27890919a9be0032eb0bfe
Subject label: N6108 Session label: ses-01  ID:  5d331cc87288c50038d14560
Subject label: N6081 Session label: ses-01  ID:  5d28c7a219a9be0030eb081e
Subject label: N6115 Session label: ses-01  ID:  5d2f8a9d7288c50038d143bb
Subject label: N6099 Session label: ses-01  ID:  5d27434d19a9be0035eb088d
Subject label: N6033 Session label: ses-01  ID:  5d23ab5919a9be0030eb075f
Subject label: N6019 Session label: ses-01  ID:  5d28e18519a9be0033eb07ab
Subject label: N6032 Session label: ses-01  ID:  5d16657f0a5781002d7d30bd
Subject label: N6104 Session label: ses-01  ID:  5d309ebd7288c50038d1446f
Subject label: N6114 Session label: ses-01  ID:  5d28f98e19a9be0032eb0cf1


Let's then focus on a specific session (project NOAH, subject N6114,  session 01)

In [19]:
ses = fw.get("5d28f98e19a9be0032eb0cf1")
print(f'Project: {fw.get(ses.parents.project).label}, Subject: {fw.get(ses.parents.subject).label}, {ses.label}')

Project: NOAH, Subject: N6114, ses-01


### Acquisitions

Within the session object, we can go another level down and list all its acquisitions using the method *acquisitions()*:

In [20]:
for acq in ses.acquisitions():
    print("acquisition:", acq.label, "ID:", acq.id)

acquisition: anat-T1w_acq-mprage ID: 5d28f9c219a9be0035eb08f1
acquisition: anat-scout_acq-aaxMPRxsag ID: 5d28f99119a9be0027eb072d
acquisition: fmap-epi_acq-siemens ID: 5d28fa3b19a9be0032eb0d01
acquisition: anat-scout_acq-aa ID: 5d28f98e19a9be0032eb0cf2
acquisition: anat-scout_acq-aaxMPRxtra ID: 5d28f99719a9be0026eb071d
acquisition: dwi_acq-dti6dxb5_dir-PA ID: 5d28fa4019a9be0032eb0d03
acquisition: anat-scout_acq-aaxMPRxcor ID: 5d28f99419a9be002eeb0853
acquisition: func-bold_task-msit_acq-mb3 ID: 5d28fa2919a9be0035eb08f3
acquisition: anat-T2w_acq-space ID: 5d29066619a9be002eeb085b
acquisition: dwi_acq-dti20dxb900 ID: 5d28fac519a9be0035eb08f9
acquisition: func-bold_task-stroop_acq-mb3 ID: 5d28faba19a9be0035eb08f7
acquisition: dwi_acq-dti6dxb5_dir-PAxTRACEW ID: 5d28fa5219a9be0033eb07b2
acquisition: func-bold_task-emoreap_acq-mb3 ID: 5d29063919a9be0033eb07b5
acquisition: dwi_acq-dti80dxb3000 ID: 5d2905bc19a9be002deb0769


for example, let's say we want to retrieve the mprage acquisition. As we saw, we can do this using its ID

In [21]:
t1_mprage = fw.get("5d28f9c219a9be0035eb08f1")

And inside our acquisitions, we finally have all the files, which in our case these are usually the DICOM images, the NIFTI images (generated from running dcm2niix) and the QA reports (generated from running MRIQC).

In [22]:
for file in t1_mprage.files:
    print("name:", file.name, "ID: ", file.id)

name: 1.3.12.2.1107.5.2.43.166114.2019071215030952973111457.0.0.0.dicom.zip ID:  29fbcc8b-d4b5-462c-8250-9bf0a0c8b96d
name: 1.3.12.2.1107.5.2.43.166114.2019071215030952973111457.0.0.0.nii.gz ID:  881a7e10-d6a1-4503-ad19-7fd188a25c58
name: 1.3.12.2.1107.5.2.43.166114.2019071215030952973111457.0.0.0_mriqc.qa.html ID:  222b0195-0a83-466d-82d3-b60420ff2aa7


### Analysis

As we said above, analysis objects are another kind of containers. Analyses are available on Projects, Subjects, Sessions and Acquisitions. These can be created uploading new files from an analysis performed locally or, as we are going to see later, by running an **analysis gear**.

In [23]:
one_sess = fw.get("5d16657f0a5781002d7d30bd")
print(f'subject: {fw.get(one_sess.parents.subject).label}, {one_sess.label}')
print("")
one_sess = one_sess.reload() # This is important and seems to be a ew
for analysis in one_sess.analyses:
    print(f'analysis label: {analysis.label}, analysis ID: {analysis.id}')

subject: N6032, ses-01

analysis label: fmriprep_no_sdc_w_reconall, analysis ID: 5e335e655c40e2054df41c1e
analysis label: fmriprep 11/06/2019 09:44:07, analysis ID: 5dc2dc3f119463002d0934e0
analysis label: fmriprep_syn_sdc_w_reconall, analysis ID: 5ee7d740a6863f017b691759


## Finding Objects

We have seen how to retrieve data objects using their ID numbers and the function *get()*. However, Flywheel allows you to access objects using paths, given that the data are organized hierarchically, as we saw. This can be done using the method *lookup()*.

(another way is using *resolve()*
https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/matlab/getting_started.html#finding-objects)

In [24]:
# The same acquisition as above using lookup
t1_mprage_lookup=fw.lookup("pgianaroslab/NOAH/N6114/ses-01/anat-T1w_acq-mprage")
print(t1_mprage.id, t1_mprage_lookup.id)

5d28f9c219a9be0035eb08f1 5d28f9c219a9be0035eb08f1


# GEARS

Gears are implemented applications in order to run tasks on data on Flywheel. These tasks may include metadata extraction, classification, quality assurance, format conversion, and analytic pipelines. 

There are two types of Gears: Utility and Analysis.

- ***Utility Gears***: These are either a conversion Gear, which takes data from one format to another, or a QA Gear, which provides quality assurance on the data. Utility Gears can also be called pre-processing Gears and can be assembled into a pipeline to run automatically. Utility gears place their **outputs in the same container as the inputs**, in contrast to Analysis Gears.

- ***Analysis Gears***: These perform advanced preprocessing and data analysis. Their **outputs are saved as a analysis object**, which are another kind of container.

*Note*: Once data are uploaded to Flywheel, data might be automatically processed according to a given Project's "Gear Rules". Rules define which Gear will be run on a given piece of data right after it is transferred to to Flywheel. In our case (NOAH and eBACH), these rules convert the DICOM into niftis and run MRIQC.

We can see the gears we can access to using the method *gears()* of the client instance:

In [25]:
for gear in fw.gears():
    print(f'Gears label: {gear.gear.label}, Gears name: {gear.gear.name}, ID: {gear.id}')

Gears label: BIDS fMRIPrep: A Robust Preprocessing Pipeline for fMRI Data, Gears name: bids-fmriprep, ID: 5fac00deb8995e1f9493c38f
Gears label: DICOM to NIfTI conversion using dcm2niix with an optional implementation of PyDeface., Gears name: dcm2niix, ID: 5f871c179e219ee28c3f88d6
Gears label: BIDS Curation, Gears name: curate-bids, ID: 5f56b20f27d54a74bda9dc61
Gears label: fMRIPREP: A Robust Preprocessing Pipeline for fMRI Data, Gears name: fmriprep, ID: 5f1eee8159341208230f01e7
Gears label: Project-specific Deidentification, Gears name: project-sorting-deid, ID: 5ea1d61fb3066d01f96e2d1d
Gears label: CMRR: Extract CMRR Physio, Gears name: extract-cmrr-physio, ID: 5e7b9a7ab68e5623377e6cc7
Gears label: QSIPREP: workflows for preprocessing and reconstructing q-space images, Gears name: qsiprep-fw, ID: 5df9653326ab57001847de88
Gears label: FreeSurfer: MBIRN Defacer for structural MRI (mri-deface v1.22), Gears name: mri-deface, ID: 5dc1c58f11946300290900c1
Gears label: SciTran: DICOM MR Cl

Once we know the id of the gear, we can define an instance of it using the method *get_gear()* in te client. Example:

In [26]:
freesurfer_gear = fw.get_gear("5d76809c9bcf05002e95fab9")

Alternatively, we could've used lookup() using the name of the gear 

In [ ]:
# Alternatively
fw.lookup('gears/freesurfer-recon-all').id

'5d76809c9bcf05002e95fab9'

We can now use this instance to run freesurfer on our data in flywheel. Before that, it is useful to know the details of the application. We can print these details using the method *print_details()* in the gear instance object:

In [27]:
freesurfer_gear.print_details()

FreeSurfer (v6.0.0): Recon-All

This gear takes an anatomical NIfTI file and performs all of the FreeSurfer cortical
  reconstruction process. Outputs are provided in a zip file and include the entire output
  directory tree from Recon-All. Configuration options exist for setting the subject ID
  and for converting outputs to NIfTI, OBJ, and CSV. FreeSurfer is a software package for
  the analysis and visualization of structural and functional neuroimaging data from
  cross-sectional or longitudinal studies. It is developed by the Laboratory for
  Computational Neuroimaging at the Athinoula A. Martinos Center for Biomedical Imaging.
  Please see https://surfer.nmr.mgh.harvard.edu/fswiki/FreeSurferSoftwareLicense for
  license information.

Name:           freesurfer-recon-all
Version:        0.3.0
Category:       analysis
Author:         Laboratory for Computational Neuroimaging <freesurfer@nmr.mgh.harvard.edu>
Maintainer:     Michael Perry <lmperry@stanford.edu>
URL:            https:

Finally, in order to run the gear we have to use the method *run()* of the gear instance object. We are not going to run it now, as we'll do this shortly below, but it's important to know that this is a function of the form *run(config, analysis_label, tags, destination, inputs)*, where the arguments are:

- config: The configuration to use, if overriding defaults. ( dict )
- analysis_label: The label of the analysis, if running an analysis gear. (str)
- tags: The list of tags to set for the job. (list)
- destination: The destination container. (object)
- inputs: The list of input containers or files. (dict)

In [28]:
# (By the way, in python you can easily access the docs appending the question "?"" to the name of the objects)
freesurfer_gear.run?

As part of the tuorial, let's try running a few gears on a given subject in order to learn how to use them

In [29]:
test_session = fw.lookup("pgianaroslab/NOAH/N6604/ses-01")

## Example 1: Run a utility gear. In this case, dcm2niix.

Interestingly (because it was totally random when I chose this subject), for this particular subject dcm2niix gear, which converts the DICOMs to niftis and runs automatically after the subject data are transfered from the scanner to the server, produces two nifti files instead of one for emotion regulation. 

This is a known issue that happens once in a while randomly, only affects the emotion regulation acquisition, and has to do with the slices in the DICOM file having a different sign in one of the metafields.

In [30]:
emoreap_acq = fw.lookup("pgianaroslab/NOAH/N6604/ses-01/func-bold_task-emoreap_acq-mb3")

In [31]:
for file in emoreap_acq.files:
    print(file.name, file.id)

1.3.12.2.1107.5.2.43.166114.2021071411373476218523878.0.0.0.dicom.zip 0c1e1fe9-e6b6-4581-91a5-0511b756e7fe
1_3_12_2_1107_5_2_43_166114_2021071411373476218523878_0_0_0_c.nii.gz d8547932-6cd2-4e9e-81da-2f053e8317d4
1_3_12_2_1107_5_2_43_166114_2021071411373476218523878_0_0_0_cHC1-7.nii.gz ab233773-0f01-46e7-960c-ebdcef33b3bf
1_3_12_2_1107_5_2_43_166114_2021071411373476218523878_0_0_0_cHC1-7_mriqc.qa.html 905288b2-3bbb-4303-8a58-9579028621f3
1_3_12_2_1107_5_2_43_166114_2021071411373476218523878_0_0_0_c_mriqc.qa.html d1bdf62d-809a-477b-9149-df9cf3f8c2a5


The way to solve this is to, after removing the old nifti files (do this better on the GUI), rerun dcm2niix using activating the option **merge_2d**.

In [32]:
dcm2niix_gear = fw.lookup('gears/dcm2niix') 
# Or using get_gear...
# dcm2niix_gear = fw.get_gear("5f871c179e219ee28c3f88d6") 

dcm2niix_gear.print_details()

DICOM to NIfTI conversion using dcm2niix with an optional implementation of PyDeface.

Implementation of Chris Rorden's dcm2niix for converting DICOM to NIfTI, with an optional
  implementation of Poldrack Lab's PyDeface to remove facial structures from NIfTI.

Name:           dcm2niix
Version:        1.0.0_1.0.20200331
Category:       converter
Author:         Flywheel
Maintainer:     Flywheel <support@flywheel.io>
URL:            https://github.com/rordenlab/dcm2niix &
                https://github.com/poldracklab/pydeface
Source:         https://github.com/flywheel-apps/dcm2niix

Inputs:
  dcm2niix_input (file, required)
    Type: dicom, parrec
    Input file for dcm2niix. This can be either a DICOM archive ('.dicom.zip'), a PAR/REC
      archive ('.parrec.zip'), or a single PAR file ('image.PAR').
  rec_file_input (file, optional)
    Type: parrec
    If dcm2niix_input is a single PAR file, the corresponding REC file ('image.REC') for
      one par/rec file pair as inputs to dcm2n

Let's create a dictionary that we will pass to the gear instance when running it, modifying activating the option **merge_2d**. 

Instead of creating the configuration dictionary from zero, we can load the default configuration of the gear using the method *get_default_config()* of the gear object instance and modify it.

In [33]:
# Default configuration
dcm2niix_gear_config = dcm2niix_gear.get_default_config()
dcm2niix_gear_config

{'anonymize_bids': True,
 'bids_sidecar': 'n',
 'coil_combine': False,
 'compress_nifti': 'y',
 'compression_level': 6,
 'convert_only_series': 'all',
 'crop': False,
 'decompress_dicoms': False,
 'filename': '%f',
 'ignore_derived': False,
 'ignore_errors': False,
 'lossless_scaling': False,
 'merge2d': False,
 'philips_scaling': True,
 'pydeface': False,
 'pydeface_cost': 'mutualinfo',
 'pydeface_nocleanup': False,
 'pydeface_verbose': False,
 'remove_incomplete_volumes': False,
 'single_file_mode': False,
 'text_notes_private': False}

We can then just modify the field **merge_2d** of this dictionary to set to True:

In [34]:
dcm2niix_gear_config['merge2d'] = True

dcm2niix_gear_config

{'anonymize_bids': True,
 'bids_sidecar': 'n',
 'coil_combine': False,
 'compress_nifti': 'y',
 'compression_level': 6,
 'convert_only_series': 'all',
 'crop': False,
 'decompress_dicoms': False,
 'filename': '%f',
 'ignore_derived': False,
 'ignore_errors': False,
 'lossless_scaling': False,
 'merge2d': True,
 'philips_scaling': True,
 'pydeface': False,
 'pydeface_cost': 'mutualinfo',
 'pydeface_nocleanup': False,
 'pydeface_verbose': False,
 'remove_incomplete_volumes': False,
 'single_file_mode': False,
 'text_notes_private': False}

In [35]:
dicom_file = "1.3.12.2.1107.5.2.43.166114.2021071411373476218523878.0.0.0.dicom.zip"
dcm2_niix_input = {'dcm2niix_input': emoreap_acq.get_file(dicom_file)}

In [36]:
dcm2niix_gear.run(config=dcm2niix_gear_config, inputs=dcm2_niix_input, destination=emoreap_acq)

'60f9c057f1f409bfb9e81b6d'

And now we should have just one nifti file:

In [37]:
emoreap_acq = emoreap_acq.reload()
for file in emoreap_acq.files:
    print(file.name, file.id)

1.3.12.2.1107.5.2.43.166114.2021071411373476218523878.0.0.0.dicom.zip 0c1e1fe9-e6b6-4581-91a5-0511b756e7fe
1_3_12_2_1107_5_2_43_166114_2021071411373476218523878_0_0_0.nii.gz ec829850-32bb-4485-b239-672f9133718a


## Example 2: Run a utility gear. In this case, bids curation.

In [38]:
bids_curation = fw.get_gear("5f56b20f27d54a74bda9dc61")

# Or alternatively using lookup
#bids_curation = fw.lookup("gears/curate-bids'") 

# Print details about the gear
bids_curation.print_details()

BIDS Curation

Use this gear to initialize BIDS filenames and attributes on all files within a given
  project.

Name:           curate-bids
Version:        1.0.0_0.9.1
Category:       classifier
Author:         Flywheel <support@flywheel.io>
Maintainer:     Flywheel <support@flywheel.io>
URL:            http://bids.neuroimaging.io/
Source:         https://github.com/flywheel-apps/curate-bids

Inputs:
  api_key (api-key, required)

Configuration:
  reset (boolean, default: False)
    Remove all BIDS info from files before curating
  entire_project (boolean, default: True)
    Run bids curation on the entire project


In [39]:
bids_curation_config = bids_curation.get_default_config()
print(bids_curation_config)

bids_curation_config['entire_project'] = False
bids_curation_config['reset'] = True

print(bids_curation_config)

{'reset': False, 'entire_project': True}
{'reset': True, 'entire_project': False}


And now we run it as we've done with dcm2niix. In this case, this gear does need inputs, so we just need to pass the configuration dictionary and the destination object:

In [41]:
bids_curation.run(config=bids_curation_config, destination=test_session)

'60f9c1b7bf0e510089e81be3'

## Example 3: Run an analysis gear. In this case, fmriprep

In [42]:
fmriprep_gear = fw.lookup('gears/fmriprep')

# Or alternatively using lookup
#fmriprep_gear = fw.get_gear("5f1eee8159341208230f01e7") 

fmriprep_gear.print_details()

fMRIPREP: A Robust Preprocessing Pipeline for fMRI Data

fmriprep is a functional magnetic resonance imaging (fMRI) data preprocessing pipeline
  that is designed to provide an easily accessible, state-of-the-art interface that is
  robust to variations in scan acquisition protocols and that requires minimal user input,
  while providing easily interpretable and comprehensive error and output reporting. It
  performs basic processing steps (coregistration, normalization, unwarping, noise
  component extraction, segmentation, skullstripping etc.) providing outputs that can be
  easily submitted to a variety of group level analyses, including task-based or resting-
  state fMRI, graph theory measures, surface or volume-based statistics, etc.

Name:           fmriprep
Version:        6.1.2_1.5.5
Category:       analysis
Author:         Poldrack Lab, Stanford University
Maintainer:     Flywheel <support@flywheel.io>
URL:            https://fmriprep.readthedocs.io/en/1.5.5/
Source:         

However, and in particular for fmriprep, gears may have different versions of the applications to run and we may be interested in running one of these versions. In the case of fmriprep, this is useful, because the latest version contains a bug, which has been already reported.

In order to find our version of the gear, the method *get_all_gears()* of the client can be useful:

In [43]:
# Let's look at the docs of this method:
fw.get_all_gears?

In [44]:
fmriprep_gears = fw.get_all_gears(filter="gear.name=fmriprep", all_versions=True)

print(f'There are {len(fmriprep_gears)} versions of fmriprep:')

for gear in fmriprep_gears:
    print(gear.gear.name, gear.gear.version, gear.id)

There are 6 versions of fmriprep:
fmriprep 6.1.2_1.5.5 5f1eee8159341208230f01e7
fmriprep 5.7.1_1.2.6-1 5d76809c9bcf05003895fabb
fmriprep 5.7.0_1.2.6-1.patch2 5d657e40e3250a0038b77f8c
fmriprep 5.7.0_1.2.6-1.patch1 5d6563bbe3250a0033b77131
fmriprep 5.7.0_1.2.6-1 5d4352e552f58b0026c65c1d
fmriprep 5.6.3_1.2.6-1 5cf6871cadf3ec00362a743c


With the idenfication number we can easily retrieve the version that we want to use. In this case, we will use the next-to-latest version:

In [45]:
fmriprep_gear_v57 = fw.get_gear("5d76809c9bcf05003895fabb")

fmriprep_gear_v57.print_details()

fMRIPREP: A Robust Preprocessing Pipeline for fMRI Data

fmriprep is a functional magnetic resonance imaging (fMRI) data preprocessing pipeline
  that is designed to provide an easily accessible, state-of-the-art interface that is
  robust to variations in scan acquisition protocols and that requires minimal user input,
  while providing easily interpretable and comprehensive error and output reporting. It
  performs basic processing steps (coregistration, normalization, unwarping, noise
  component extraction, segmentation, skullstripping etc.) providing outputs that can be
  easily submitted to a variety of group level analyses, including task-based or resting-
  state fMRI, graph theory measures, surface or volume-based statistics, etc.

Name:           fmriprep
Version:        5.7.1_1.2.6-1
Category:       analysis
Author:         Poldrack Lab, Stanford University
Maintainer:     Flywheel <support@flywheel.io>
URL:            https://fmriprep.readthedocs.io/en/1.2.6-1/
Source:     

In [46]:
config_fmriprep = fmriprep_gear_v57.get_default_config()
config_fmriprep

{'FREESURFER_LICENSE': 'timothyv@andrew.cmu.edu 28995 *CH4FQzSJfAZc FSUWkGWrILQCg',
 'anat_only': False,
 'aroma_melodic_dimensionality': 200,
 'bold2t1w_dof': 6,
 'cifti_output': False,
 'echo_idx': '',
 'fmap_bspline': False,
 'fmap_no_demean': True,
 'force_bbr': False,
 'force_no_bbr': False,
 'force_syn': False,
 'fs_no_reconall': False,
 'ignore': '',
 'ignore_aroma_denoising_errors': False,
 'intermediate_files': '',
 'intermediate_folders': '',
 'longitudinal': False,
 'medial_surface_nan': False,
 'no_submm_recon': False,
 'no_track': False,
 'output_space': 'template fsaverage5',
 'save_intermediate_work': False,
 'save_outputs': False,
 'skip_bids_validation': True,
 'skull_strip_fixed_seed': False,
 'skull_strip_template': 'OASIS',
 't2s_coreg': False,
 'task_id': '',
 'template': 'MNI152NLin2009cAsym',
 'template_resampling_grid': 'native',
 'use_aroma': False,
 'use_syn_sdc': False}

In [47]:
config_fmriprep['use_syn_sdc'] = True
config_fmriprep['ignore'] = 'fieldmaps'
config_fmriprep

{'FREESURFER_LICENSE': 'timothyv@andrew.cmu.edu 28995 *CH4FQzSJfAZc FSUWkGWrILQCg',
 'anat_only': False,
 'aroma_melodic_dimensionality': 200,
 'bold2t1w_dof': 6,
 'cifti_output': False,
 'echo_idx': '',
 'fmap_bspline': False,
 'fmap_no_demean': True,
 'force_bbr': False,
 'force_no_bbr': False,
 'force_syn': False,
 'fs_no_reconall': False,
 'ignore': 'fieldmaps',
 'ignore_aroma_denoising_errors': False,
 'intermediate_files': '',
 'intermediate_folders': '',
 'longitudinal': False,
 'medial_surface_nan': False,
 'no_submm_recon': False,
 'no_track': False,
 'output_space': 'template fsaverage5',
 'save_intermediate_work': False,
 'save_outputs': False,
 'skip_bids_validation': True,
 'skull_strip_fixed_seed': False,
 'skull_strip_template': 'OASIS',
 't2s_coreg': False,
 'task_id': '',
 'template': 'MNI152NLin2009cAsym',
 'template_resampling_grid': 'native',
 'use_aroma': False,
 'use_syn_sdc': True}

In contrast to utility gears, when running an analysis gear, we need to pass it label:

In [48]:
analysis_label = 'fmriprep_syn_sdc_w_reconall'

In [49]:
fmriprep_gear_v57.run(config=config_fmriprep, 
                      analysis_label=analysis_label, 
                      inputs=[], 
                      destination=test_session)

'60f9c3a3bf0e510089e81be4'